In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 1000  # Number of epochs to train for.
latent_dim = 1024  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'kor-eng/kor.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True, return_sequences=True)
encoder2 = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder2(encoder(encoder_inputs))

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder2(decoder(decoder_inputs, initial_state=encoder_states))

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s2.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder2_outputs, state_h2, state_c2 = decoder2(decoder(decoder_inputs, initial_state=[state_h, state_c]))
decoder_states = [state_h2, state_c2]
decoder_outputs = decoder_dense(decoder2_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Using TensorFlow backend.


Number of samples: 799
Number of unique input tokens: 69
Number of unique output tokens: 632
Max sequence length for inputs: 124
Max sequence length for outputs: 54
Train on 639 samples, validate on 160 samples
Epoch 1/1000
639/639 [==============================] - 344s 539ms/step - loss: 1.5363 - val_loss: 2.1542
Epoch 2/1000
639/639 [==============================] - 337s 527ms/step - loss: 1.2015 - val_loss: 2.0809
Epoch 3/1000
639/639 [==============================] - 337s 527ms/step - loss: 1.1065 - val_loss: 2.0830
Epoch 4/1000
639/639 [==============================] - 339s 530ms/step - loss: 1.1114 - val_loss: 2.1068
Epoch 5/1000
639/639 [==============================] - 343s 537ms/step - loss: 1.0835 - val_loss: 2.1216
Epoch 6/1000
639/639 [==============================] - 345s 539ms/step - loss: 1.0706 - val_loss: 2.0454
Epoch 7/1000
639/639 [==============================] - 350s 548ms/step - loss: 1.0686 - val_loss: 2.0542
Epoch 8/1000
639/639 [=========================

639/639 [==============================] - 362s 567ms/step - loss: 0.1485 - val_loss: 2.3532
Epoch 76/1000
639/639 [==============================] - 367s 574ms/step - loss: 0.1342 - val_loss: 2.3668
Epoch 77/1000
639/639 [==============================] - 362s 567ms/step - loss: 0.1264 - val_loss: 2.3715
Epoch 78/1000
639/639 [==============================] - 362s 566ms/step - loss: 0.1206 - val_loss: 2.3821
Epoch 79/1000
639/639 [==============================] - 363s 568ms/step - loss: 0.1237 - val_loss: 2.3941
Epoch 80/1000
639/639 [==============================] - 362s 567ms/step - loss: 0.1135 - val_loss: 2.4135
Epoch 81/1000
639/639 [==============================] - 362s 567ms/step - loss: 0.1115 - val_loss: 2.4463
Epoch 82/1000
639/639 [==============================] - 362s 566ms/step - loss: 0.1054 - val_loss: 2.4389
Epoch 83/1000
639/639 [==============================] - 362s 567ms/step - loss: 0.1046 - val_loss: 2.4696
Epoch 84/1000
639/639 [============================

639/639 [==============================] - 389s 609ms/step - loss: 0.0048 - val_loss: 2.7233
Epoch 152/1000
639/639 [==============================] - 371s 581ms/step - loss: 0.0107 - val_loss: 2.7807
Epoch 153/1000
639/639 [==============================] - 369s 578ms/step - loss: 0.0347 - val_loss: 2.6977
Epoch 154/1000
639/639 [==============================] - 391s 611ms/step - loss: 0.0049 - val_loss: 2.7193
Epoch 155/1000
639/639 [==============================] - 399s 624ms/step - loss: 0.0040 - val_loss: 2.7276
Epoch 156/1000
639/639 [==============================] - 385s 602ms/step - loss: 0.0214 - val_loss: 2.7197
Epoch 157/1000
639/639 [==============================] - 399s 624ms/step - loss: 0.0086 - val_loss: 2.7379
Epoch 158/1000
639/639 [==============================] - 402s 629ms/step - loss: 0.0030 - val_loss: 2.7457
Epoch 159/1000
639/639 [==============================] - 440s 688ms/step - loss: 0.0145 - val_loss: 2.7778
Epoch 160/1000
639/639 [===================

639/639 [==============================] - 387s 605ms/step - loss: 0.0011 - val_loss: 2.9613
Epoch 227/1000
639/639 [==============================] - 397s 622ms/step - loss: 0.0063 - val_loss: 2.9643
Epoch 228/1000
639/639 [==============================] - 383s 600ms/step - loss: 0.0041 - val_loss: 2.9416
Epoch 229/1000
639/639 [==============================] - 381s 597ms/step - loss: 0.0045 - val_loss: 2.9516
Epoch 230/1000
639/639 [==============================] - 386s 605ms/step - loss: 8.6778e-04 - val_loss: 2.9374
Epoch 231/1000
639/639 [==============================] - 365s 572ms/step - loss: 6.0261e-04 - val_loss: 2.9467
Epoch 232/1000
639/639 [==============================] - 363s 568ms/step - loss: 0.0041 - val_loss: 2.9443
Epoch 233/1000
639/639 [==============================] - 368s 577ms/step - loss: 0.0021 - val_loss: 2.9293
Epoch 234/1000
639/639 [==============================] - 364s 570ms/step - loss: 0.0021 - val_loss: 2.9441
Epoch 235/1000
639/639 [===========

639/639 [==============================] - 360s 563ms/step - loss: 1.0991e-04 - val_loss: 3.1825
Epoch 301/1000
639/639 [==============================] - 360s 564ms/step - loss: 0.0117 - val_loss: 3.1212
Epoch 302/1000
639/639 [==============================] - 360s 563ms/step - loss: 4.3545e-04 - val_loss: 3.1106
Epoch 303/1000
639/639 [==============================] - 359s 563ms/step - loss: 3.0696e-04 - val_loss: 3.1189
Epoch 304/1000
639/639 [==============================] - 360s 563ms/step - loss: 6.0708e-04 - val_loss: 3.1232
Epoch 305/1000
639/639 [==============================] - 360s 564ms/step - loss: 0.0035 - val_loss: 3.2169
Epoch 306/1000
639/639 [==============================] - 360s 564ms/step - loss: 0.0073 - val_loss: 3.1032
Epoch 307/1000
639/639 [==============================] - 360s 564ms/step - loss: 0.0029 - val_loss: 3.0888
Epoch 308/1000
639/639 [==============================] - 360s 563ms/step - loss: 0.0018 - val_loss: 3.0567
Epoch 309/1000
639/639 [===

639/639 [==============================] - 388s 606ms/step - loss: 5.9003e-07 - val_loss: 3.2465
Epoch 374/1000
639/639 [==============================] - 386s 604ms/step - loss: 5.8006e-07 - val_loss: 3.2474
Epoch 375/1000
639/639 [==============================] - 383s 600ms/step - loss: 5.7113e-07 - val_loss: 3.2483
Epoch 376/1000
639/639 [==============================] - 419s 656ms/step - loss: 5.6208e-07 - val_loss: 3.2491
Epoch 377/1000
639/639 [==============================] - 395s 618ms/step - loss: 5.5335e-07 - val_loss: 3.2500
Epoch 378/1000
639/639 [==============================] - 400s 625ms/step - loss: 5.4473e-07 - val_loss: 3.2507
Epoch 379/1000
639/639 [==============================] - 388s 607ms/step - loss: 5.3650e-07 - val_loss: 3.2514
Epoch 380/1000
639/639 [==============================] - 381s 597ms/step - loss: 5.2880e-07 - val_loss: 3.2522
Epoch 381/1000
639/639 [==============================] - 382s 597ms/step - loss: 5.2134e-07 - val_loss: 3.2531
Epoch 3